In [2]:
# imports
import pandas as pd
import os
import requests
import sqlite3
from sqlite3 import Error
import math

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [3]:
foursquare_id = os.environ["FOURSQUARE_API_KEY"]

def get_venues_FS(ll):
    url = "https://api.foursquare.com/v3/places/search"

    headers = {
        "Accept": "application/json",
        "Authorization": foursquare_id
    }

    data = {'ll': ll,
            'radius': ('1000'),
            'categories': ('13000'),
            'fields': 'fsq_id,name,rating,stats,popularity,price,categories'
    }

    response = requests.get(url, headers=headers,params=data)
    return response
    

In [33]:
#Reference dataframe for FourSquare category IDs
categoryFS = pd.DataFrame.from_dict({
    13000:['all'], 13001:['Bagel Shop'], 13002:['Bakery'], 13003:['Bar'], 13004:['Ski Bar'],
    13005:['Beach Bar'], 13006:['Beer Bar'], 13007:['Beer Garden'], 13008:['Champagne Bar'],
    13009:['Cocktail Bar'], 13010:['Dive Bar'], 13011:['Gay Bar'], 13012:['Hookah Bar'],
    13013:['Hotel Bar'], 13014:['Ice Bar'], 13015:['Karaoke Bar'], 13016:['Lounge'],
    13017:['Piano Bar'], 13018:['Pub'], 13019:['Rooftop Bar'], 13020:['Sake Bar'], 13021:['Speakeasy'],
    13022:['Sports Bar'], 13023:['Tiki Bar'], 13024:['Whisky Bar'], 13025:['Wine Bar'],
    13028:['Breakfast Spot'], 13029:['Brewery'], 13032:['Cafes, Coffee, and Tea Houses'],
    13033:['Bubble Tea Shop'], 13034:['Café'], 13035:['Coffee Shop'], 13063:['Pet Café'],
    13036:['Tea Room'], 13037:['Cafeteria'], 13038:['Cidery'], 13040:['Dessert Shop'],
    13041:['Creperie'], 13042:['Cupcake Shop'], 13043:['Donut Shop'], 13044:['Frozen Yogurt Shop'],
    13045:['Gelato Shop'], 13046:['Ice Cream Parlor'], 13047:['Pastry Shop'], 13048:['Pie Shop'],
    13049:['Diner'], 13050:['Distillery'], 13051:['Fish and Chips Shop'], 13052:['Food Court'], 
    13053:['Food Stand'], 13054:['Food Truck'], 13059:['Juice Bar'],13065:['Restaurant'],
    13068:['American Restaurant'],13099:['Chinese Restaurant'],13338:['Seafood Restaurant']
    }, orient='index', columns=['Type'])

In [13]:
categoryFS.head()

,Type
13000,all
13001,Bagel Shop
13002,Bakery
13003,Bar
13004,Ski Bar


Parse through the response to get the POI details you want (rating, name, location, etc)

In [5]:
respDataFS = get_venues_FS("43.397145,-80.360949")


In [6]:

def parseResults_FS(respData):
    #Results - Buisness list
    #dataC - (string)Unique buisness id(fsq_id): (int)category type(See categoryFS for table lookup)
    results= []
    dataC = []
    #Data to Parse
    #fsq_id - Unique ID (string)
    #name   - Buisness Name (string)
    #rating - (float) Values 1-10
    #popularity - (float) Values 0-1
    #price - (int) Values 1=Cheap, 2=Moderate, 3=Expensive, 4=Very Expensive
    #categories - Multiple (int) 
    for r in respData.json()['results']:
        dataL = {}
        if 'fsq_id' in r:
            dataL['id'] = r['fsq_id']    
        else:
            print("Missing id")
            continue
        if 'name' in r:
            dataL['name']=  r['name']
        if 'rating' in r:
            try:
                dataL['rating'] = float(r['rating'])
            except:
                print(f"Parse Error: {dataL['id']} - Rating")
        if 'stats' in r:
            if ('total_ratings' in r['stats']):
                try:
                    dataL['total_ratings'] = int(r['stats']['total_ratings'])
                except:
                    print(f"Parse Error: {dataL['id']} - Stats")
        if 'popularity' in r:
            try:
                dataL['popularity'] = float(r['popularity'])
            except Exception as e:
                print(f"Parse Error: {dataL['id']} - Popularity - {r['popularity']}")
        if 'price' in r:
            try:
                dataL['price'] = int(r['price'])
            except:
                print(f"Parse Error: {dataL['id']} - Price")
        if 'categories' in r:
            for c in r['categories']:            
                try:
                    dataC.append({'fsq_id': dataL['id'],'id': int(c['id'])})
                except:
                    print(f"Parse Error: {dataL['id']} - Categories")
        results.append(dataL)
    

    return pd.DataFrame(results),pd.DataFrame(dataC)

In [7]:
fsDF,fsCategoriesDF = parseResults_FS(respDataFS)

In [37]:
fsDF

,id,name,rating,total_ratings,popularity,price
0,4b81e84ef964a52086c330e3,Fiddle & Firkin,8.1,18.0,0.973292,1.0
1,4c40e3f0e26920a19da15de7,Rising Dough Bakery,7.8,17.0,0.990267,1.0
2,4bf5bd9f9abec9b6239624e8,Argyle Arms,7.4,10.0,0.883784,2.0
3,4c4b0122c668e21e9f4274f9,Cambridge Family Restaurant,7.3,8.0,0.905394,2.0
4,4b9c2404f964a5203a4e36e3,Dairy Queen,7.1,5.0,0.965993,1.0
5,4c94c70738dd8cfa4ce8cb62,Central Hotel,NaN,NaN,0.937431,NaN
6,4e8782c78231468c5871de70,Preston Chipwagon,NaN,NaN,0.551127,1.0
7,4c57030b2308be9a3a2d586c,Mr. Sub,NaN,NaN,0.830021,2.0
8,4cbb24c19552b60cc838de8b,Cambridge Restaurant,NaN,NaN,0.824112,NaN
9,4b7c7470f964a52088942fe3,Shanghai Restaurant,NaN,NaN,0.021609,NaN


Put your parsed results into a DataFrame

In [35]:
pd.merge(pd.merge(fsCategoriesDF,fsDF, left_on='fsq_id',right_on='id')[['fsq_id', 'id_x','name']],
        categoryFS, left_on='id_x', right_index=True, how='left')[['fsq_id', 'name', 'Type']]


,fsq_id,name,Type
0,4b81e84ef964a52086c330e3,Fiddle & Firkin,Pub
1,4b81e84ef964a52086c330e3,Fiddle & Firkin,Restaurant
2,4c40e3f0e26920a19da15de7,Rising Dough Bakery,Bakery
3,4bf5bd9f9abec9b6239624e8,Argyle Arms,Bar
4,4bf5bd9f9abec9b6239624e8,Argyle Arms,Diner
5,4bf5bd9f9abec9b6239624e8,Argyle Arms,American Restaurant
6,4c4b0122c668e21e9f4274f9,Cambridge Family Restaurant,Restaurant
7,4b9c2404f964a5203a4e36e3,Dairy Queen,Ice Cream Parlor
8,4c94c70738dd8cfa4ce8cb62,Central Hotel,Bar
9,4c94c70738dd8cfa4ce8cb62,Central Hotel,Restaurant


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [39]:
yelp_id = os.environ["YELP_API_KEY"]

def get_venues_Yelp(latitude, longitude):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Accept": "application/json",
        "Authorization": 'Bearer ' + yelp_id
    }

    data = {'latitude': latitude,
            'longitude': longitude,
            'radius': ('1000'),
            'categories': ('food,bars,All')
    }

    response = requests.get(url, headers=headers,params=data)
    #print (response.url)
    return response
    

Parse through your result and get POI details

In [40]:
respDataYelp = get_venues_Yelp("43.397145","-80.360949")

Put your parsed results into a DataFrame

In [41]:
def parseResults_Yelp(respData):
    #Results - Buisness list
    #dataC - (string)Unique buisness id(fsq_id): (string)category type
    results= []
    dataC = []
    #Data to Parse
    #id - Unique ID (string)
    #name   - Buisness Name (string)
    #rating - (float) Values 1-5
    #review_count - (int) Number of reviews
    #price - (string) Values $,$$,$$$,$$$$
    #categories - Multiple (string) 
    for r in respData.json()['businesses']:
        dataL = {}
        if 'id' in r:
            dataL['id'] = r['id']    
        else:
            print("Missing id")
            continue
        if 'name' in r:
            dataL['name']=  r['name']
        if 'rating' in r:
            try:
                dataL['rating'] = float(r['rating'])
            except:
                print(f"Parse Error: {dataL['id']} - Rating")
        if 'review_count' in r:
            try:
                dataL['review_count'] = int(r['review_count'])
            except:
                print(f"Parse Error: {dataL['id']} - Review Count")
        if 'price' in r:
            dataL['price'] = r['price']
        if 'categories' in r:
            for c in r['categories']:
                dataC.append({'id': dataL['id'],'Category': c['title']})                
        results.append(dataL)

    return pd.DataFrame(results),pd.DataFrame(dataC)

In [42]:
yelpDF,yelpCateDF = parseResults_Yelp(respDataYelp)

In [43]:
yelpDF

,id,name,rating,review_count,price
0,vPJzugWveS48LYBBJvLCWg,The Fiddle & Firkin,4.5,13,$$
1,OVLCf_KQQVQDKzriKyMRVA,Rising Dough Bakery,3.5,3,NaN
2,X2JsurdEue0K99-YE8Cflw,Burns Howff Scottish Pub,3.5,2,NaN
3,yRR7zw1ipOP85BiFLMcp0Q,Shanghai Restaurant,2.5,2,NaN
4,zMs3m2Q-7btEU-s41462xQ,Boutique Baker,2.5,4,$$
5,HS43WOAu5vXqTlL2s9RQnA,Sidelines Sports Bar & Grill,4.0,1,NaN


In [45]:
pd.merge(yelpCateDF, yelpDF, on='id')[['id', 'name', 'Category']]

,id,name,Category
0,vPJzugWveS48LYBBJvLCWg,The Fiddle & Firkin,Pubs
1,vPJzugWveS48LYBBJvLCWg,The Fiddle & Firkin,American (Traditional)
2,OVLCf_KQQVQDKzriKyMRVA,Rising Dough Bakery,Bakeries
3,X2JsurdEue0K99-YE8Cflw,Burns Howff Scottish Pub,Pubs
4,yRR7zw1ipOP85BiFLMcp0Q,Shanghai Restaurant,Chinese
5,yRR7zw1ipOP85BiFLMcp0Q,Shanghai Restaurant,Imported Food
6,zMs3m2Q-7btEU-s41462xQ,Boutique Baker,Bakeries
7,zMs3m2Q-7btEU-s41462xQ,Boutique Baker,Desserts
8,HS43WOAu5vXqTlL2s9RQnA,Sidelines Sports Bar & Grill,Sports Bars


# Google (stretch)

Use the same process as the first two APIs

In [47]:
google_id = os.environ["GOOGLE_API_KEY"]
def get_venues_Google(ll):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    headers = {
        "Accept": "application/json"
    }

    data = {'key': google_id,
            'location': ll,
            'radius': ('1000'),
            'type': 'bar|restaurant'

    }

    response = requests.get(url, headers=headers,params=data)
    return response
    

In [48]:
respDataGoogle = get_venues_Google("43.397145,-80.360949")

In [49]:
def parseResults_Google(respData):
    #Results - Buisness list
    #dataC - (string)Unique buisness id(fsq_id): (string)category type
    results= []
    dataC = []
    #Data to Parse
    #place_id - Unique ID (string)
    #name   - Buisness Name (string)
    #rating - (float) Values 1-5
    #user_ratings_total - (int) Review Count
    #price_level - (int) Values 0 - Free, 1 - Inexpensive, 2 - Moderate
    #                           3 - Expensive, 4 - Very Expensive
    #types - *Categories* Multiple (string) 
    i = 0
    for r in respData.json()['results']:
        dataL = {}
        if 'place_id' in r:
            dataL['place_id'] = r['place_id']
        else:
            print("Missing id")
            continue
        if 'name' in r:
            dataL['name']=  r['name']
        if 'rating' in r:
            try:
                dataL['rating'] = float(r['rating'])
            except:
                print(f"Parse Error: {dataL['place_id']} - Rating")
        if 'user_ratings_total' in r:
            try:
                dataL['user_ratings_total'] = int(r['user_ratings_total'])
            except:
                print(f"Parse Error: {dataL['place_id']} - Review Count")
        if 'price_level' in r:
            try:
                dataL['price_level'] = int(r['price_level'])
            except:
                print(f"Parse Error: {dataL['place_id']} - Price Level")
        if 'types' in r:
            for c in r['types']:
                dataC.append({'place_id': dataL['place_id'],'Category': c})                
        results.append(dataL)

    return pd.DataFrame(results),pd.DataFrame(dataC)

In [50]:
googleDF,googleCateDF = parseResults_Google(respDataGoogle)

In [51]:
googleDF

,place_id,name,rating,user_ratings_total,price_level
0,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,4.4,260,2.0
1,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,4.4,558,2.0
2,ChIJtfE8PvCJK4gRZEXIt4owgTo,The Hopper,2.7,26,1.0
3,ChIJHRPY7--JK4gRw58ONIT-_fY,Sidelines Sports Bar & Grill,3.8,208,1.0
4,ChIJH0h0JF-JK4gR5rsg2qBBsRM,Wave Maker Craft Brewery & Taproom,4.9,197,NaN


In [52]:
pd.merge(googleCateDF, googleDF, on='place_id')[['place_id', 'name', 'Category']]

,place_id,name,Category
0,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,bar
1,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,restaurant
2,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,food
3,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,point_of_interest
4,ChIJ8WNx3vWJK4gR-7ucQoV2gbw,Argyle Arms,establishment
5,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,bar
6,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,restaurant
7,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,food
8,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,point_of_interest
9,ChIJtX9JHfGJK4gRm-mGqE3tPr0,Fiddle And Firkin,establishment


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [107]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [206]:
create_foursquare_table = """
CREATE TABLE IF NOT EXISTS fsVenues (
  id TEXT PRIMARY KEY,
  name TEXT NOT NULL,
  rating REAL,
  popularity REAL,
  price INTEGER
);
"""
create_foursquare_categories_lookup = """
CREATE TABLE IF NOT EXISTS fsCategoryLookup (
  id INTEGER PRIMARY KEY,
  type TEXT NOT NULL
);
"""

create_foursquare_categories = """
CREATE TABLE IF NOT EXISTS fsCategories (
  id TEXT NOT NULL,
  categoryid INTEGER NOT NULL,
  FOREIGN KEY(id) REFERENCES fsVenues(id),
  FOREIGN KEY(categoryid) REFERENCES fsCategoryLookup(id)
);
"""

create_yelp_table = """
CREATE TABLE IF NOT EXISTS yVenues (
  id TEXT PRIMARY KEY,
  name TEXT NOT NULL,
  rating REAL,
  review_count INTEGER,
  price TEXT
);
"""
create_yelp_categories = """
CREATE TABLE IF NOT EXISTS yCategories (
  id TEXT NOT NULL,
  type TEXT NOT NULL,
  FOREIGN KEY(id) REFERENCES yVenues(id)
);
"""
create_google_table = """
CREATE TABLE IF NOT EXISTS gVenues (
  id TEXT PRIMARY KEY,
  name TEXT NOT NULL,
  rating REAL,
  user_ratings_total INTEGER,
  price_level INTEGER
);
"""
create_google_categories = """
CREATE TABLE IF NOT EXISTS gCategories (
  id TEXT NOT NULL,
  type TEXT NOT NULL,
  FOREIGN KEY(id) REFERENCES gVenues(id)
);
"""




In [108]:
connection = create_connection('..\data\BuisnessDB.sqlite')

Connection to SQLite DB successful


In [208]:
#CREATE DATABASE TABLES
execute_query(connection, create_foursquare_table)
execute_query(connection, create_foursquare_categories_lookup)
execute_query(connection, create_foursquare_categories)
execute_query(connection, create_yelp_table)
execute_query(connection, create_yelp_categories)
execute_query(connection, create_google_table)
execute_query(connection, create_google_categories)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [ ]:
#FourSquare insert into database
for index,row in categoryFS.iterrows():
    sql = f"INSERT INTO fsCategoryLookup(id, type) \
VALUES({index}, '{row['Type']}');"
    execute_query(connection, sql)

for index,row in fsDF.iterrows():
    id = row['id']
    name = row['name']

    if math.isnan(row['rating']):
        rating = 'NULL'
    else:
        rating = row['rating']

    if math.isnan(row['popularity']):
        popularity = 'NULL'
    else:
        popularity = row['popularity']

    if math.isnan(row['price']):
        price = 'NULL'
    else:
        price = row['price']

    sql = f"INSERT INTO fsVenues(id, name, rating, popularity, price) \
VALUES('{id}', '{name}', {rating}, \
{popularity}, {price});"
    execute_query(connection, sql)

for index,row in fsCategoriesDF.iterrows():
    sql = f"INSERT INTO fsCategories(id, categoryid) \
VALUES('{row['fsq_id']}', '{row['id']}');"
    execute_query(connection, sql)

In [ ]:
#Yelp insert into database
for index,row in yelpDF.iterrows():
    id = row['id']
    name = row['name']
    if math.isnan(row['rating']):
        rating = 'NULL'
    else:
        rating = row['rating']
    if math.isnan(row['review_count']):
        review_count = 'NULL'
    else:
        review_count = row['review_count']
    
    price = str(row['price'])
    if price == 'nan':
        price = 'NULL'
    else:
        price = "'" + price + "'"
    sql = f"INSERT INTO yVenues(id, name, rating, review_count, price) \
VALUES('{id}', '{name}', {rating}, {popularity}, {price});"
    execute_query(connection, sql)
    #print(sql)

for index,row in yelpCateDF.iterrows():
    sql = f"INSERT INTO yCategories(id, type) \
VALUES('{row['id']}', '{row['Category']}');"
    execute_query(connection, sql)
    #print(sql)

In [214]:
#Google insert into database
for index,row in googleDF.iterrows():
    id = row['place_id']
    name = row ['name']
    if math.isnan(row['rating']):
        rating = 'NULL'
    else:
        rating = row['rating']
    if math.isnan(row['user_ratings_total']):
        user_ratings_total = 'NULL'
    else:
        user_ratings_total = row['user_ratings_total']
    
    if math.isnan(row['price_level']):
        price = 'NULL'
    else:
        price = row['price_level']

    sql = f"INSERT INTO gVenues(id, name, rating, user_ratings_total, price_level) \
VALUES('{id}', '{name}', {rating}, {user_ratings_total}, {price});"
    execute_query(connection, sql)

for index,row in googleCateDF.iterrows():
    sql = f"INSERT INTO gCategories(id, type) \
VALUES({index}, '{row['Category']}');"
    execute_query(connection, sql)


Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [109]:
print (execute_read_query(connection, "SELECT * FROM gVenues"))

[('ChIJ8WNx3vWJK4gR-7ucQoV2gbw', 'Argyle Arms', 4.4, 260, 2), ('ChIJtX9JHfGJK4gRm-mGqE3tPr0', 'Fiddle And Firkin', 4.4, 558, 2), ('ChIJtfE8PvCJK4gRZEXIt4owgTo', 'The Hopper', 2.7, 26, 1), ('ChIJHRPY7--JK4gRw58ONIT-_fY', 'Sidelines Sports Bar & Grill', 3.8, 206, 1), ('ChIJH0h0JF-JK4gR5rsg2qBBsRM', 'Wave Maker Craft Brewery & Taproom', 4.9, 197, None)]


Get the top 10 restaurants according to their rating

In [114]:
#Clean venue names
fsDF['name'] = fsDF['name'].str.replace('The ', '').str.replace('And', '&')
googleDF['name'] = googleDF['name'].str.replace('The ', '').str.replace('And', '&')
yelpDF['name'] = yelpDF['name'].str.replace('The ', '').str.replace('And', '&')

#Merge Dataframes 
ratingDF = pd.merge(fsDF[fsDF['rating'] > 0][['name','rating']],
            yelpDF[yelpDF['rating'] > 0][['name', 'rating']],
            on='name', how='outer')
ratingDF = pd.merge(ratingDF,
            googleDF[googleDF['rating'] > 0][['name', 'rating']],
            on='name', how='outer')
ratingDF = ratingDF.rename(columns={'rating_x': 'FS Rating (1-10)', 'rating_y': 'Yelp Rating (1-5)',
                                'rating': 'Google Rating (1-5)'})

In [115]:
#Create two new columns Total (max 30) and Avg Rating
#Total (max 30) - Totals adjusted ratings to rating out of 10
#Avg Rating     - Averages adjusted ratings, ignores NULL values
newCol = []
for index,row in ratingDF.iterrows():
    data = {}
    i = 0
    total = 0
    data['name'] = row['name']
    if row['FS Rating (1-10)'] > 0:
        total += row['FS Rating (1-10)']
        i += 1
    if row['Yelp Rating (1-5)'] > 0:
        total += row['Yelp Rating (1-5)'] * 2
        i += 1
    if row['Google Rating (1-5)'] > 0:
        total += row['Google Rating (1-5)'] * 2
        i += 1
    
    data['Total (max 30)'] = total
    data['Avg Rating'] = round(total/i,2)
    newCol.append(data)

#Merge new columns onto Dataframe
ratingDF = pd.merge(ratingDF, pd.DataFrame(newCol), on='name')


In [117]:
ratingDF.sort_values(by='Avg Rating', ascending=False)

,name,FS Rating (1-10),Yelp Rating (1-5),Google Rating (1-5),Total (max 30),Avg Rating
10,Wave Maker Craft Brewery & Taproom,NaN,NaN,4.9,9.8,9.80
0,Fiddle & Firkin,8.1,4.5,4.4,25.9,8.63
2,Argyle Arms,7.4,NaN,4.4,16.2,8.10
8,Sidelines Sports Bar & Grill,NaN,4.0,3.8,15.6,7.80
1,Rising Dough Bakery,7.8,3.5,NaN,14.8,7.40
3,Cambridge Family Restaurant,7.3,NaN,NaN,7.3,7.30
4,Dairy Queen,7.1,NaN,NaN,7.1,7.10
5,Burns Howff Scottish Pub,NaN,3.5,NaN,7.0,7.00
9,Hopper,NaN,NaN,2.7,5.4,5.40
6,Shanghai Restaurant,NaN,2.5,NaN,5.0,5.00


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).